In [25]:
%pip install torchvision.transform

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement torchvision.transform (from versions: none)
ERROR: No matching distribution found for torchvision.transform


In [26]:
%pip install cv2

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2


In [1]:
from torchvision.transforms import transforms

In [2]:
import cv2

image = cv2.imread("Latin/A/5a0d5cfd92c94.png")
image.shape

(278, 278, 3)

In [3]:
from torchvision.io import read_image

image = read_image("Latin/A/5a0d5cfd92c94.png")
image.shape

torch.Size([4, 278, 278])

In [4]:
import torch

a = torch.rand(5)

In [5]:
f_t = transforms.Lambda(lambda x: x * 2)
f_t(a)
transforms.ToPILImage()

ToPILImage()

In [6]:
f_t1 = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((124, 124)),
    transforms.PILToTensor()
])
f_t1(image)
transforms.ToPILImage(image)

ToPILImage(mode=tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]], dtype=torch.uint8))

In [7]:
import pathlib
from torch.utils.data import Dataset

In [8]:
class LatinDataset(Dataset):
  def __init__(self, path_dataset: pathlib.Path):
    self.path_dataset = path_dataset
    self.data_list = [x for x in self.path_dataset.glob("**/*") if x.is_file()]
    self.data_class = list(set(x.parent for x in self.data_list))
    self.transform_func = transforms.Compose([
      transforms.ToPILImage(),
      transforms.Grayscale(),
      transforms.Resize((124, 124)),
      transforms.PILToTensor()
  ])

  def __len__(self):
    return len(self.data_list)

  def __getitem__(self, index):
    img_path = self.data_list[index]
    img_label = torch.zeros(len(self.data_class))
    img_label[self.data_class.index(img_path.parent)] = 1.0
    img_tensor = read_image(img_path)
    return self.transform_func(img_tensor), img_label


In [9]:
import torch.nn as nn

In [10]:
import torch

a = torch.rand((2, 1, 1, 28))
a.size()

torch.Size([2, 1, 1, 28])

In [11]:
a.view(-1, 28).size()

torch.Size([2, 28])

In [12]:
class CVModel(nn.Module):
  def __init__(self):
    self.conv_1 = nn.Conv2d(1, 32, (3, 3))
    self.conv_2 = nn.Conv2d(32, 64, (3, 3))
    self.pool_1 = nn.MaxPool2d((2, 2))
    self.conv_3 = nn.Conv2d(64, 128, (3, 3))
    self.conv_4 = nn.Conv2d(128, 128, (3, 3))
    self.pool_2 = nn.MaxPool2d((2, 2))
    self.glob_pool = nn.MaxPool2d((28, 28))
    self.linear = nn.Linear(128, 26)
    self.softmax = nn.Softmax()

  def forward(self, X):
    res = self.conv_1(X)
    res = self.conv_2(res)
    res = self.pool_1(res)
    res = self.conv_3(res)
    res = self.conv_4(res)
    res = self.pool_2(res)
    res = self.glob_pool(res)
    res = self.linear(res.view(-1, 128))
    return self.softmax(res)
